## Tutorial 3 - Training Neural Networks

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

## Network Initialization
Initializing neural network layers are very important for training. The required functions for network initialization are located in `torch.nn.init`.

There are a few common initialization methods used:
- `torch.nn.init.zeros_(tensor)`: Initializes the tensor to all zeros.
- `torch.nn.init.normal_(tensor, mean, std)`: Initializes from the normal distribution $\mathcal{N}(\mu, \sigma^2)$.
- `torch.nn.init.xavier_uniform_(tensor, g=1)`: Initializes from the uniform distribution $\mathcal{U}(-a, a)$, where $a$ is given by the formula $g \times \sqrt{\frac{6}{f_{in} + f_{out}}}$. Here, $g$ is called the gain, $f_{in}$ is the input dimensionality of the weight matrix, and $f_{out}$ is the output dimensionality.
- `torch.nn.init.xavier_normal_(tensor, g=1)`: Initializes from the normal distribution $\mathcal{N}(0, \sigma^{2})$, where $\sigma = g \times \sqrt{\frac{2}{f_{in} + f_{out}}}$.

By default, PyTorch initializes Linear layers using `kaiming_uniform` initialization. You can read more about this in the below link.

All these initializations and other choices can be found at the [PyTorch initialization documentation](https://pytorch.org/docs/stable/nn.init.html).

# Dropout

Dropout is a regularization technique used to prevent **overfitting** in neural networks.  
During training, dropout randomly "drops" (sets to zero) a fraction $p$ of the neurons' outputs in a layer at each forward pass.  
This forces the network to not rely too heavily on any specific neuron and instead learn more robust and generalizable feature representations.

Mathematically, for a given layer output vector **h**, dropout creates a masked version:

$h' = m \odot h, \quad \text{where } m_i \sim \text{Bernoulli}(1-p)$

Here, $m$ is a binary mask sampled independently for each neuron, and $\odot$ denotes element-wise multiplication.

In PyTorch, dropout is implemented using:
```python
nn.Dropout(p)
```
where p is the parameter that controls the fraction of masking $p$.

## Normalization
To use the different types of normalization, you can use the following:
- `torch.nn.BatchNorm(1d/2d/3d)`: Applies batch normalization on data with the given number of dimensions (excluding batch dim)
- `torch.nn.GroupNorm()`: Applies group normalization
- `torch.nn.InstanceNorm(1d/2d/3d)`: Applies instance normalization on the data
- `torch.nn.LayerNorm`: Applies LayerNorm

**Batch Normalization**  
Normalizes activations across the batch and feature dimensions. It computes the mean and variance for each feature over the entire batch and rescales values accordingly. This helps the network train faster and acts as a mild regularizer. Works best when batch sizes are not too small.

$$
y = \gamma \times \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta
$$
Here, $\gamma, \beta$ are parameters learned by the normalization layer, which allows for scaling the mean and variance to whatever is best.

**Layer Normalization**  
Normalizes all features within a single sample, making it independent of the batch size. It is commonly used in Transformers and RNNs, where batch statistics may not be stable or meaningful. The mean and variance are based on the individual samples in the batch, instead of across the whole batch.

**RMS Normalization**
While LayerNorm is very effective for many applications, it does suffer from one drawback: the need to calculate the mean and standard deviation of the inputs. Many recent models, especially Transformers (which you will learn about later this semester), use a faster normalization method instead, called RMSNorm.

$$
y = \gamma \times \frac{x}{\sqrt{\frac{1}{d} \sum_{i=1}^{d} x_i^2 + \epsilon}}
$$

Notice here how we do not require any calculation of the mean or variance, instead just normalizing based on the input itself, and additionally we do not have any shift factor $\beta$. However, we do keep the scale factor $\gamma$ which is learned.

RMSNorm allows for trading off speed (since we don't compute means or variances) for potential performance (since we don't center the activations around 0, and we don't allow for additional shift factors). Thus, it is a design decision of which to use.

**Instance Normalization**  
Normalizes each feature map for every sample individually. It’s often used in style transfer and image generation tasks, as it removes instance-specific contrast while preserving the overall style.

Instance Normalization uses the same equation as LayerNorm, with the only difference being how the mean and variance is calculated (LayerNorm takes the mean and variance over all elements within a sample, while InstanceNorm handles different channels separately, for example).

**Group Normalization**  
Splits the feature channels into groups and normalizes within each group for every sample. It’s useful when batch sizes are small or vary across training steps, providing a middle ground between BatchNorm and LayerNorm.

Again, the equation is the same, the only change is how the mean and variance is calculated (for example over groups of channels).

<img src="https://media.geeksforgeeks.org/wp-content/uploads/20250114183648606652/What-is-Group-Normalization_.jpg" width="600">


For this tutorial, you should only need to use BatchNorm.

To use weight normalization, you will need to use a different approach. If `layer` is a layer that you want to apply weight normalization to, you should use `layer = torch.nn.utils.parametrizations.weight_norm(layer)`. This will always apply weight norm.

### Exercise 1: Creating a Properly Initialized Network
Below is the starter code for a basic MLP. You should implement the following:
- `3` linear layers, with a ReLU activation between each layer.
- The first layer should have input dim `input_dim` and output dim 64. The second layer should have both input and output dims as 64. The last layer should have input dim 64 and output dim `num_classes`.
- Initialize all of the linear layers using xavier uniform initialization. (Note: To access the weights of a layer, you need to use `layer.weight`. To access the bias of a layer, you need to use `layer.bias`).
- Initialize all bias terms to all zeros.
- Use Batch Norm in between each of the layers (Optional).
- Use Dropout in between each layer (Optional).

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.linear1 = nn.Linear(input_size, 64)
        self.linear2 = nn.Linear(64, 64)
        self.linear3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
        nn.init.xavier_uniform_(self.linear1.weight)
        nn.init.xavier_uniform_(self.linear2.weight)
        nn.init.xavier_uniform_(self.linear3.weight)
        if self.linear1.bias is not None:
            nn.init.zeros_(self.linear1.bias)
        if self.linear2.bias is not None:
            nn.init.zeros_(self.linear2.bias)
        if self.linear3.bias is not None:
            nn.init.zeros_(self.linear3.bias)

    def forward(self, x):
        x = x.view(x.shape[0], -1) # flattening to shape: B * L
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        out = self.linear3(x)

        return out

In [ ]:
input_size = 784
num_classes = 10
learning_rate = 1e-3
batch_size = 8
num_epochs = 3

In [ ]:
my_transforms = transforms.Compose(
    [
        transforms.ToTensor(),                    # Finally converts PIL image to tensor
        #transforms.Resize((36, 36)),             # Resizes (32,32) to (36,36)
        #transforms.RandomCrop((32, 32)),         # Takes a random (32,32) crop
        #transforms.RandomRotation(degrees=45),   # Perhaps a random rotation from -45 to 45 degrees
        #transforms.RandomHorizontalFlip(p=0.5),  # Flips the image horizontally with probability 0.5
        #transforms.RandomVerticalFlip(p=0.05),   # Flips image vertically with probability 0.05
        #transforms.Normalize((0.5,), (0.5,))     # Normalize
    ]
)

In [ ]:
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=my_transforms, download=True)
test_dataset = datasets.MNIST(root="dataset/", train=False, transform=my_transforms, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

## Optimizers and Schedulers
As covered in the previous tutorial, the optimizers are located in `torch.optim`. Commonly used optimizers are:
- `torch.optim.Adam`
- `torch.optim.AdamW`
- `torch.optim.SGD`
- `torch.optim.RMSprop`

You can find the full list of optimizers and their details at `https://pytorch.org/docs/stable/optim.html`.

Learning rate schedulers are located in `torch.optim.lr_scheduler`. Commonly used ones are:
- `torch.optim.lr_scheduler.LinearLR`: Linearly changes the learning rate according to the number of steps
- `torch.optim.lr_scheduler.ExponentialLR`: Apply exponential decay to the learning rate. If `gamma` is the exponential factor, then the current learning rate is given by `curr_lr = start_lr * (gamma)^num_steps`.
- `torch.optim.lr_scheduler.CosineAnnealingLR`: Implements cosine annealing. A more complicated scheduler, refer to the paper [SGDR: Stochastic Gradient Descent with Warm Restarts](https://arxiv.org/abs/1608.03983) for more details.
- `torch.optim.lr_scheduler.ReduceLROnPlateau`: Monitors a given metric, and reduces the learning rate whenever the metric stops improving.

All learning rate schedulers take as input the optimizer, as well as any other hyperparameters. If `scheduler` is the learning rate scheduler, then you can update the learning rate by calling `scheduler.step()`.


### Weight Decay

Weight decay is a form of **regularization** that helps prevent overfitting by discouraging large weight values.  
It works by adding a small penalty proportional to the magnitude of the weights to the loss function during optimization.  
This causes the optimizer to slightly shrink the weights on every update, keeping them smaller and more stable.

With **weight decay**, an extra term proportional to the current weight value is added to the gradient:
$w = w - \eta \cdot (\nabla_w L + \lambda w)$
where $\lambda$ is the **weight decay coefficient**.


To use weight decay, you can use the keyword `weight_decay=` when creating the optimizer.

### L1/L2 Regularization
To apply some type of regularization to the model, you can use:
- `sum(p.pow(2).sum() for p in model.parameters())`: L2 normalization
- `sum(p.abs().sum() for p in model.parameters())`: L1 normalization

### Exercise 2: Optimizers and Schedulers
Experiment with different choices of optimizers and learning rate schedulers to see how good the performance can be. Initialize an optimizer here called `optimizer`, and a learning rate scheduler called `scheduler`.

In [ ]:
model = MLP(input_size, num_classes).to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
def check_accuracy(loader, model, device):
    """
    Check accuracy of our trained model given a loader and a model

    Parameters:
        loader: torch.utils.data.DataLoader
            A loader for the dataset you want to check accuracy on
        model: nn.Module
            The model you want to check accuracy on

    Returns:
        acc: float
            The accuracy of the model on the dataset given by the loader
    """

    num_correct = 0
    num_samples = 0
    model.eval()

    # We don't need to keep track of gradients here so we wrap it in torch.no_grad()
    with torch.no_grad():
        # Loop through the data
        for x, y in loader:

            # Move data to device
            x = x.to(device=device)
            y = y.to(device=device)

            # Get to correct shape
            x = x.reshape(x.shape[0], -1)

            # Forward pass
            scores = model(x)
            _, predictions = scores.max(1)

            # Check how many we got correct
            num_correct += (predictions == y).sum()

            # Keep track of number of samples
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples

### Exercise 3: Training Loop
Here the basics of the training loop are provided. Implement the forward and backward passes, optimizer and learning rate scheduler steps.

In [ ]:
model.train()
for epoch in range(num_epochs):
    losses = []
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # TODO: Add code here
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        mean_loss = sum(losses) / len(losses)
    scheduler.step()
    print(f"Epoch: [{epoch+1}/{num_epochs}] | Loss: {mean_loss:.3f}")

# Check accuracy on training & test to see how good our model
print(f"Accuracy on training set: {check_accuracy(train_loader, model, device)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model, device)*100:.2f}")

100%|██████████| 7500/7500 [00:21<00:00, 356.53it/s]


Epoch: [1/3] | Loss: 0.228


100%|██████████| 7500/7500 [00:21<00:00, 342.48it/s]


Epoch: [2/3] | Loss: 0.083


100%|██████████| 7500/7500 [00:21<00:00, 350.00it/s]


Epoch: [3/3] | Loss: 0.067
Accuracy on training set: 98.03
Accuracy on test set: 97.24


### Exercise 4: Improving Performance
Modify the optimizer, learning rate scheduler, type of normalization, dropout percentage, weight initialization, learning rate, and regularization, and see how high you can get the accuracy.